In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import layers, models
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import cv2


In [2]:
def read_video_cv2(path, n_frames=1000):
    cap = cv2.VideoCapture(path)
    all = []
    i = 0
    while cap.isOpened() and i < n_frames:
        ret, frame = cap.read()
        if not ret:
            break
        arr = np.array(frame)
        all.append(arr)
        i += 1

    return np.array(all)

In [3]:
path = './Real_Life_Violence_Dataset/'
NFileList = os.listdir(path+'NonViolence')
VFileList = os.listdir(path+'Violence')

NFileList = [x for x in NFileList if ('mp4' or 'avi') in x]
VFileList = [x for x in VFileList if ('mp4' or 'avi') in x]

FileList = NFileList + VFileList

In [4]:
classification = {'NV' : 0, 'V' : 1}

videos = []
labels = []

for i, file in enumerate(FileList):
    label, _ = file.split('_') # 예시 aa_bc_01_02.jpg
    code, ext = _.split('.') # 예시 02.jpg

    if label == "NV":
        code = "NonViolence/"
    elif label == "V":
        code = "Violence/"
    else:
        continue

    # 이미지 불러오기
    video = read_video_cv2(path+code+file)
    video.resize(video.shape[0], 32, 32, 3)
    video = video / 255

    if video.dtype != np.float64:
        continue

    if video.shape[0] < 55:
        continue

    videos.append(video[0:55])
    labels.append(classification[label])

In [5]:
X = np.array(videos)
Y = np.array(labels)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)

In [7]:
def convolutional_model():
    model = models.Sequential()
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=X_train.shape[1:]))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

model = convolutional_model()

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10)

Epoch 1/10
49/49 [==============================] - 210s 4s/step - loss: 0.7199 - accuracy: 0.5173 - val_loss: 0.6618 - val_accuracy: 0.5385
Epoch 2/10
49/49 [==============================] - 177s 4s/step - loss: 0.6530 - accuracy: 0.6187 - val_loss: 0.6112 - val_accuracy: 0.6538
Epoch 3/10
49/49 [==============================] - 163s 3s/step - loss: 0.5734 - accuracy: 0.7125 - val_loss: 0.6033 - val_accuracy: 0.6256
Epoch 4/10
49/49 [==============================] - 176s 4s/step - loss: 0.5618 - accuracy: 0.6977 - val_loss: 0.5711 - val_accuracy: 0.7256
Epoch 5/10
49/49 [==============================] - 174s 4s/step - loss: 0.5172 - accuracy: 0.7490 - val_loss: 0.5200 - val_accuracy: 0.7795
Epoch 6/10
49/49 [==============================] - 176s 4s/step - loss: 0.4627 - accuracy: 0.7914 - val_loss: 0.5255 - val_accuracy: 0.7692
Epoch 7/10
49/49 [==============================] - 175s 4s/step - loss: 0.4226 - accuracy: 0.8113 - val_loss: 0.5123 - val_accuracy: 0.7769
Epoch 8/10
49

In [8]:
model.save('../model.h5')

c:\Users\SOOM\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
m = tf.keras.models.load_model('../model.h5')

In [13]:
X[0].shape

(55, 32, 32, 3)

In [14]:
X[0:1].shape

(1, 55, 32, 32, 3)

In [16]:
np.array([[1], [1], [1]]).shape

(3, 1)

In [ ]:
a[3][1]

In [12]:
m.predict(X[0:1])

1/1 [==============================] - 0s 133ms/step


array([[0.678309]], dtype=float32)

In [19]:
print(X[0:1].shape)

(1, 55, 32, 32, 3)
